In [351]:
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import pandas as pd
import numpy as np
from numpy import mat
import random as rnd  #This module implements pseudo-random number generators for various distributions.
import scipy.stats as stats  #contains  probability distributions as well as  statistical functions.
import scipy.optimize as opt
from scipy.sparse import diags
import json as json
import matplotlib as mpl
from numpy import exp
from numpy import log
from matplotlib import pyplot as plt
import time

from IPython.core.pylabtools import figsize
from IPython.display import display
from IPython.core.display import HTML
%matplotlib inline

rnd.seed(2)

figsize(12, 8)
pd.set_option("display.precision", 3)
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

In [352]:
# read the simulated data
lin_dataframe=pd.read_csv('Lin_Dataset.csv')
df=lin_dataframe

In [363]:
s=70 # Our chosen states, you should discretzie your mileage to state first.
beta=0.75 # discount facotr
threshold=1e-6
theta0=[20,0.5]


In [354]:
# cost functions denote the n*1 vector of all values the cost function can assume, ie. c=(c(1,theta), ..., c(n, theta))

def lin_cost(s, params):
    try:
        theta1_1 = params  
        return s*theta1_1
    except ValueError:
        raise ValueError
        print("Wrong number of parameters specified: expected 2, got {}".format(len(params)))
        

def quad_cost(s, params):
    try:
        theta1_1, theta1_2 = params # single common denotes a two-tuple
        return s*theta1_1 + (s**2)*theta1_2
    except ValueError:
        raise ValueError
        print("Wrong number of parameters specified: expected 2, got {}".format(len(params)))
        
def exp_cost(s, params):
    try:
        theta1_1, = params
        return np.exp(s*theta1_1)
    except ValueError:
        raise ValueError
        print("Wrong number of parameters specified: expected 1, got {}".format(len(params)))

def log_cost(s, params):
    try:
        theta1_1, theta1_2 = params
        return np.log(theta1_1 + s*theta1_2)
    except ValueError:
        raise ValueError
        print("Wrong number of parameters specified: expected 2, got {}".format(len(params)))        


### Mileage Transition Definition
For this reason, we will adopt this **first** approach, and assumes that the increase in mileage follows a truncated normal distribution bounded at [0, 15000]:

$$
(M_{t+1} - M_{t}) \sim \mathcal{N}_{Trunc}(6000, 4000)
$$


In [355]:
#set up the transition probabilty 
lower, upper = 0, 15000
mu, sigma = 6000, 4000 #The dist. is defined over standard normal and we need to normalized it as follows
mileage_dist = stats.truncnorm((lower - mu) / sigma, (upper - mu) / sigma, loc=mu, scale=sigma) 

In [356]:
p_x0 = mileage_dist.cdf(5000)
p_x1 = mileage_dist.cdf(10000) - p_x0
p_x2 = 1 - p_x1 - p_x0
p = (p_x0, p_x1, p_x2)

### Discretize the state space

The mileage is our only state variable. We discrete it by a discete inteval of 5000 miles. The maximum state in our data is possilbe to be less than the total number of state.

###  Construct the state transition probability matrix



In [357]:
def build_sparse(S, p):
    out = diags(p, range(len(p)), shape=(S, S)).toarray()
    out[-2:,-1] = [1 - p[1], 1] # place absorbing state x the lower corner
    return out

In [358]:
#upper case P is matrix, lower case p is the transition probability
P=build_sparse(s,p)
P

array([[ 0.3632,  0.4778,  0.159 , ...,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.3632,  0.4778, ...,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.3632, ...,  0.    ,  0.    ,  0.    ],
       ..., 
       [ 0.    ,  0.    ,  0.    , ...,  0.3632,  0.4778,  0.159 ],
       [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.3632,  0.5222],
       [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  1.    ]])

## fixed points and mle

In [371]:
def loglike(theta):
    tr=theta[0] #total replacement cost
    theta1=theta[1] # cost paramters
    c=[lin_cost(x,theta1) for x in range(0,s)]
    c=np.array(c)
    c.shape=(s,1)  # only 1 dimention array use shape, or you will mix up the column and rows

    # compute the expected values, find the fixed points
    
    ev=np.zeros((s,1))
    # expected value for not choosing
    achieved = True
    k=0
    kev=max(-c+beta*ev)
    ev1=kev+mat(P)*mat(log(exp(-c+beta*ev-kev)+exp(-tr-c[0]+beta*ev[0]-kev)))  #check the dimentions
    ev=kev+mat(P)*mat(log(exp(-c+beta*ev1-kev)+exp(-tr-c[0]+beta*ev1[0]-kev)))  #check the dimentions
    
    t0 = time.time()
    achieved=True
    
    while abs(ev-ev1).max() > threshold:
        kev=max(-c+beta*ev)
        ev1=kev+mat(P)*mat(log(exp(-c+beta*ev-kev)+exp(-tr-c[0]+beta*ev[0]-kev)))  #check the dimentions
        ev=kev+mat(P)*mat(log(exp(-c+beta*ev1-kev)+exp(-tr-c[0]+beta*ev1[0]-kev)))  #check the dimentions
        k += 1
        if k == 5000: 
            achieved = False
            break
    
    if achieved: 
            print("Convergence achieved in {} iterations".format(k))
    else:
            print("CM could not converge! Mean difference = {:.6f}".format((ev-ev1).mean()) 
                 )
    t1 = time.time()
    total = t1-t0
    print('time:', total)

    # likelihood
    pk=1/(1+exp(c-beta*ev-tr-c[0]+beta*(ev[1])))
    prob_ev=np.hstack((pk,1-pk))
    prob=prob_ev[df.iloc[:,1],df.iloc[:,0]] # no need to minus 1, because the state in range(0,s) starting from 0
    ll=-np.sum(log(prob))
    
    return ll   

In [362]:
loglike([20,0.5])

Convergence achieved in 27 iterations
time: 0.039999961853


159415.98113015221

In [378]:
loglike([ 18.5998,   0.7023])

Convergence achieved in 27 iterations
time: 0.0339999198914


16331.300562856653

In [372]:
res = opt.minimize(loglike, theta0, method='BFGS', tol=1e-6)

Convergence achieved in 27 iterations
time: 0.0410001277924
Convergence achieved in 27 iterations
time: 0.0199999809265
Convergence achieved in 27 iterations
time: 0.0209999084473
Convergence achieved in 27 iterations
time: 0.0230000019073
Convergence achieved in 27 iterations
time: 0.0320000648499
Convergence achieved in 27 iterations
time: 0.0150001049042
Convergence achieved in 27 iterations
time: 0.0159997940063
Convergence achieved in 27 iterations
time: 0.0150001049042
Convergence achieved in 27 iterations
time: 0.0160000324249
Convergence achieved in 27 iterations
time: 0.0360000133514
Convergence achieved in 27 iterations
time: 0.0179998874664
Convergence achieved in 27 iterations
time: 0.0149998664856
Convergence achieved in 29 iterations
time: 0.0150001049042
Convergence achieved in 29 iterations
time: 0.0160000324249
Convergence achieved in 29 iterations
time: 0.0149998664856
Convergence achieved in 29 iterations
time: 0.0310001373291
Convergence achieved in 27 iterations
ti

In [375]:
res

      fun: 16331.300530123204
 hess_inv: array([[ 0.0489,  0.002 ],
       [ 0.002 ,  0.0001]])
      jac: array([ 0.    , -0.0006])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 153
      nit: 9
     njev: 36
   status: 2
  success: False
        x: array([ 18.5998,   0.7023])

In [369]:
res2 = opt.minimize(loglike2, theta0, method='BFGS', tol=1e-6)

Convergence achieved in 27 iterations
time: 0.00499987602234
Convergence achieved in 27 iterations
time: 0.00499987602234
Convergence achieved in 27 iterations
time: 0.00399994850159
Convergence achieved in 27 iterations
time: 0.010999917984
Convergence achieved in 30 iterations
time: 0.00299978256226
Convergence achieved in 30 iterations
time: 0.00300002098083
Convergence achieved in 30 iterations
time: 0.00300002098083
Convergence achieved in 30 iterations
time: 0.00399994850159
Convergence achieved in 27 iterations
time: 0.00300002098083
Convergence achieved in 27 iterations
time: 0.00300002098083
Convergence achieved in 27 iterations
time: 0.00300002098083
Convergence achieved in 27 iterations
time: 0.00300002098083
Convergence achieved in 27 iterations
time: 0.00300002098083
Convergence achieved in 27 iterations
time: 0.0
Convergence achieved in 27 iterations
time: 0.0
Convergence achieved in 27 iterations
time: 0.0
Convergence achieved in 30 iterations
time: 0.0
Convergence achie

C:\Users\leland\Anaconda2\lib\site-packages\ipykernel\__main__.py:41: RuntimeWarning: divide by zero encountered in log



Convergence achieved in 29 iterations
time: 0.00499987602234
Convergence achieved in 29 iterations
time: 0.00600004196167
Convergence achieved in 29 iterations
time: 0.00499987602234
Convergence achieved in 29 iterations
time: 0.00699996948242
Convergence achieved in 25 iterations
time: 0.0019998550415
Convergence achieved in 25 iterations
time: 0.00400018692017
Convergence achieved in 25 iterations
time: 0.00399994850159
Convergence achieved in 25 iterations
time: 0.00599980354309
Convergence achieved in 22 iterations
time: 0.00200009346008
Convergence achieved in 22 iterations
time: 0.0019998550415
Convergence achieved in 22 iterations
time: 0.00300002098083
Convergence achieved in 22 iterations
time: 0.00200009346008
Convergence achieved in 22 iterations
time: 0.00200009346008
Convergence achieved in 22 iterations
time: 0.0
Convergence achieved in 22 iterations
time: 0.0
Convergence achieved in 22 iterations
time: 0.0
Convergence achieved in 20 iterations
time: 0.0
Convergence achi

In [370]:
res2

      fun: 32443.935882368765
 hess_inv: array([[ 0.0002,  0.    ],
       [ 0.    ,  0.    ]])
      jac: array([ 0.0005,  0.0039])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 342
      nit: 11
     njev: 82
   status: 2
  success: False
        x: array([ 2.3508,  0.0065])

In [306]:
#define initial values for test purpose
theta=[-2,0.5]
tr=theta[0] #total replacement cost
theta1=theta[1] # cost paramters
#test
s=40


In [307]:
c=[lin_cost(x,theta1) for x in range(0,s)]
c=np.array(c)
c.shape=(s,1)  # only 1 dimention array use shape, or you will mix up the column and rows


### Solve for value functions

In [308]:
ev=np.zeros((s,1))
# expected value for not choosing
k=0
kev=max(-c+beta*ev)
#print('kev', kev)
#print()
ev1=kev+mat(P)*mat(log(exp(-c+beta*ev-kev)+exp(-tr-c[0]+beta*ev[0]-kev)))  #check the dimentions
#print('ev1',ev1)
#print()
ev=kev+mat(P)*mat(log(exp(-c+beta*ev1-kev)+exp(-tr-c[0]+beta*ev1[0]-kev)))  #check the dimentions
#print('ev:',ev)

In [309]:
t0 = time.time()

while abs(ev-ev1).max() > threshold:
    kev=max(-c+beta*ev)
    ev1=kev+mat(P)*mat(log(exp(-c+beta*ev-kev)+exp(-tr-c[0]+beta*ev[0]-kev)))  #check the dimentions
    ev=kev+mat(P)*mat(log(exp(-c+beta*ev1-kev)+exp(-tr-c[0]+beta*ev1[0]-kev)))  #check the dimentions

    """
    print(k)
    print(ev)
    print()
    print()
    print()
    """


    k += 1
    if k == 1000:
        achieved = False
        break
if not suppr_output:
    if achieved:
        print("Convergence achieved in {} iterations".format(k))
    else:
        print("CM could not converge! Mean difference = {:.6f}".format(
                                                            (ev-ev1).mean())
                                                                              )
t1 = time.time()
total = t1-t0
print('time:', total)

Convergence achieved in 14 iterations
time: 0.010999917984


In [173]:
'''
#A slower contraction mapping
t0 = time.time()

while abs(ev-ev1).max() > threshold:
    ev1=ev
    ev=P*log(exp(-c+beta*ev1)+exp(-tr-c[0]+beta*ev1[0]))
    print(k)
    print(ev)
    print()
    print()
    print()


    k += 1
    if k == 1000:
        achieved = False
        break
if not suppr_output:
    if achieved:
        print("Convergence achieved in {} iterations".format(k))
    else:
        print("CM could not converge! Mean difference = {:.6f}".format(
                                                            (EV_new-EV).mean())
                                                                              )
t1 = time.time()
total = t1-t0
total

np.allclose(a, b) # compare results of two mappping
'''

u'\n#A slower contraction mapping\nt0 = time.time()\n\nwhile abs(ev-ev1).max() > threshold:\n    ev1=ev\n    ev=P*log(exp(-c+beta*ev1)+exp(-tr-c[0]+beta*ev1[0]))\n    print(k)\n    print(ev)\n    print()\n    print()\n    print()\n\n\n    k += 1\n    if k == 1000:\n        achieved = False\n        break\nif not suppr_output:\n    if achieved:\n        print("Convergence achieved in {} iterations".format(k))\n    else:\n        print("CM could not converge! Mean difference = {:.6f}".format(\n                                                            (EV_new-EV).mean())\n                                                                              )\nt1 = time.time()\ntotal = t1-t0\ntotal\n\nnp.allclose(a, b) # compare results of two mappping\n'

In [310]:
pk=1/(1+exp(c-beta*ev-tr-c[0]+beta*(ev[1])))

In [311]:
pk1=1-pk

In [312]:
prob_ev=np.hstack((pk,pk1))

In [313]:
prob_ev=np.array(prob_ev) # matrix, array, dataframe, difference

In [314]:
prob=prob_ev[df.iloc[:,1]-1,df.iloc[:,0]]

In [315]:
ll=-np.sum(log(prob))
ll

474402.75467622897

In [350]:
c[0]

array([ 0.])